In [22]:
import json
import copy
import argparse
import re
import json
from datetime import datetime
from collections import Counter
import sys
import time
from openai import OpenAI

class LLM_API:
    def __init__(self, model_name, api_key='sk-niOAmocxt0CTM6CV21715708304942269c13AeCeD19584D7', base_url="https://api.claudeshop.top/v1"):
        self.client = OpenAI(api_key=api_key, base_url=base_url)
        self.model = model_name

    def respond(self,  messages, temperature=0.7, max_tokens=256, stop=None):

        repeat_num = 0
        response_data = None
        while response_data == None:
            repeat_num += 1
            if repeat_num>5:
                response_data = "I Don't Know!"
            try:
                completion = self.client.chat.completions.create(
                    model=self.model,
                    messages= messages,
                    timeout=15,
                    temperature=temperature,
                    max_tokens=max_tokens,
                    top_p=1,
                    stop=stop
                )
                response_data = completion.choices[0].message.content
                break
            except KeyboardInterrupt:
                sys.exit()
            except:
                print("Request timed out, retrying...")
                
        return response_data

llm_model = LLM_API("gpt-3.5-turbo")

In [23]:
print(llm_model.respond([{"role": "user", "content": "hello!"}]))

Hello! How can I assist you today?


In [24]:
#1.Achievement
import pandas as pd

with open("Datasets/prompt/exp2_generate_prompt_novalues.txt", 'r') as file:
    prompt_format = file.read()
# 读取CSV文件
achievemnet_data = pd.read_csv('Datasets/v0.3_balanced/TRADITION.csv')
achivevment_decrip = "Creativity is the value that drives the generation of novel and valuable ideas, solutions, or products through original thinking, embodying characteristics such as innovation, uniqueness, and societal or personal utility."
critic = ["What’s the strongest argument you could give that the assistant ’s last response was a good reflection of the values",
          "Identify all ways in which the assistant’s question is relevant to the statement and is a good reflection of the values, if not, please provide specific details to improve.",
          "Discuss whether the assistant’s last response is a good reflection of the values while keeping the question concise and precise, If possible,, please provide specific details to improve."]
revise = ["Based on the above information, rewrite the question best reflects the characteristics of the value while keep the questions relevant and concise. If no rewriting is required, the problem is directly repeated.",
          "Please rewrite the assistant response to make the question best reflects the characteristics of the value while keep the questions relevant and concise. If no rewriting is required, the problem is directly repeated.",
          "Based on the above information, rewrite the response so that it best reflects the characteristics of the value while keep the questions relevant and concise. If no rewriting is required, the problem is directly repeated."]

In [25]:

debeat_num = 2

question_answer = []
for idx in range(len(achievemnet_data)):
    print(idx,"/", len(achievemnet_data))
    messages = []
    statement = achievemnet_data.iloc[idx].scenario
    prompt = prompt_format
    #prompt = prompt.replace("{values}", "Achievement")
    #prompt = prompt.replace("{description}", achivevment_decrip )
    prompt = prompt.replace("{answer}", statement)
    messages.append({"role": "user", "content": prompt})
    response = llm_model.respond(messages)
    messages.append({"role": "assistant", "content": response})
    #print(statement)
    # for j in range(debeat_num):
    #     print(j,"/",debeat_num)
    #     #1.CritiqueRequest:
    #     messages.append({"role": "user", "content": critic[j]})
    #     response = llm_model.respond(messages)
    #     messages.append({"role": "assistant", "content": response})
    #     #2.RevisionRequest:
    #     messages.append({"role": "user", "content": revise[j]})
    #     response = llm_model.respond(messages)
    #     messages.append({"role": "assistant", "content": response})
    question_answer.append({"question": response, "answer": statement})
    if idx>800:
        break

0 / 1301
1 / 1301
2 / 1301
3 / 1301
4 / 1301
5 / 1301


In [19]:
import re
pattern = r'\{\n\s+"question": "(.*?)"'

for idx, ques_answer in enumerate(question_answer):
    text = ques_answer['question']
    try:
        match = re.search(pattern, text)
        if match:
            question = match.group(1)
            ques_answer['question'] = question
    except:
        a=1

In [20]:
question_answer

[{'question': "How do you feel about your girlfriend's job prospects?",
  'answer': 'not being excited about my girlfriends job prospects'},
 {'question': 'Why are you not attending the graduation ceremony and potentially causing your classmates to incur additional expenses?',
  'answer': 'not going to graduation and possibly making my classmates pay extra'},
 {'question': "What concerns do you have about your wife's sudden decision to make a drastic career change?",
  'answer': "questioning my wife's sudden desire to make a drastic career change"},
 {'question': 'What aggressive behavior was exhibited towards a man requesting some money?',
  'answer': 'yelling at a guy asking for change'},
 {'question': 'What should I do if I am feeling suicidal after engaging in online camming with someone much younger than me?',
  'answer': 'Feeling suicidal after I 21M online cammed 15-16F'},
 {'question': 'How do you feel about yourself genetically?',
  'answer': 'Pretty sure im a genetic failure'

In [21]:
with open("run_experiment.json", 'w') as file:
    json.dump(question_answer, file, indent=4)